In [1]:
from imputationLibrary import decompose, forwardFilling, hotDeck, meanImputation, movingAverage, splineInterpolation, randomSampleImputation
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy import signal
from sklearn import preprocessing
from datetime import datetime, timedelta
from tsmoothie.smoother import *

In [2]:
df = pd.read_csv(r'output\/CompleteDengue.csv',index_col=0, parse_dates=True)
df_train = df.loc['1990-04-30':'2004-06-03'].dropna(how='all', axis=1) #Selecting training data
df_test = df.loc['2004-06-10':, df_train.columns] #Selecting test data
period = 7
df_filled = df_train.fillna(0.000001)
df_filled_test = df_test.fillna(0.000001)

In [3]:
df_decomposed_resid = pd.DataFrame()
df_decomposed_trend = pd.DataFrame()
df_decomposed_seasonal = pd.DataFrame()

df_decomposed_resid_test = pd.DataFrame()
df_decomposed_trend_test = pd.DataFrame()
df_decomposed_seasonal_test = pd.DataFrame()

df_nan = df_train.isna()
df_nan_test = df_test.isna()

In [4]:
def plot_ac(df, name):
    normalized = preprocessing.scale([np.array(df.fillna(0))])
    corr = signal.correlate(normalized, normalized, mode='full')
    plt.plot(corr[0], 'o-', markersize=2)
    plt.axhline(2/np.sqrt(len(df)), ls=':')
    plt.axhline(-2/np.sqrt(len(df)), ls=':')
    plt.title(name + ' auto-correlation')
    plt.show()

In [7]:
def decompose_and_plot(ts, flag_plot = False):
    ts_decomposed, decomp_type =  decompose.decompose(ts, period)
    print(decomp_type)
    if flag_plot:
        ts_decomposed.plot()
        plt.show()
        print("Resid is white noise? ", nature.isWhiteNoise(ts_decomposed.resid))
        print("Resid is seasonal noise? ", nature.isSeasonal(ts_decomposed.resid))
        print("Resid is trended noise? ", nature.isTrended(ts_decomposed.resid, period))
        print("Resid is seasonal and trended noise? ", nature.isTrendedAndSeasonal(ts_decomposed.resid, period))
        plot_ac(ts_decomposed.resid, "Resid")
        plot_ac(ts_decomposed.trend, "Trend")
        plot_ac(ts_decomposed.seasonal, "Seasonal")
    return ts_decomposed.resid, ts_decomposed.trend, ts_decomposed.seasonal, decomp_type
    

In [8]:
for i in range(0,30):
    decomp_type_dict_train = {}
    decomp_type_dict_test = {}
    for col in df_filled.columns:
        resid, trend, seasonal, decomp_type = decompose_and_plot(df_filled.loc[:,col])
        decomp_type_dict_train[col] = decomp_type
        df_decomposed_resid[col] = resid
        df_decomposed_trend[col] = trend
        df_decomposed_seasonal[col] = seasonal

    for col in df_filled_test.columns:
        resid, trend, seasonal, decomp_type = decompose_and_plot(df_filled_test.loc[:,col])
        decomp_type_dict_test[col] = decomp_type
        df_decomposed_resid_test[col] = resid
        df_decomposed_trend_test[col] = trend
        df_decomposed_seasonal_test[col] = seasonal

    df_final_resid = df_decomposed_resid.mask(df_nan == True, np.nan)
    df_final_trend = df_decomposed_trend.mask(df_nan == True, np.nan)
    df_final_seasonal = df_decomposed_seasonal.mask(df_nan == True, np.nan)

    df_final_resid_test = df_decomposed_resid_test.mask(df_nan_test == True, np.nan)
    df_final_trend_test = df_decomposed_trend_test.mask(df_nan_test == True, np.nan)
    df_final_seasonal_test = df_decomposed_seasonal_test.mask(df_nan_test == True, np.nan)

    df_white_noise_train, df_white_noise_test =  meanImputation.input(df_final_resid, df_final_resid_test)
    df_seasonal_train, df_seasonal_test = splineInterpolation.input(df_final_seasonal, df_final_seasonal_test)
    df_trended_train, df_trended_test = forwardFilling.input(df_final_trend, df_final_trend_test)

    df_final_train = pd.DataFrame(index = df_filled.index, columns = df_filled.columns)
    df_final_test = pd.DataFrame(index = df_filled_test.index, columns = df_filled_test.columns)
    for col in df_filled.columns:
        if decomp_type_dict_train[col] == "additive":
            df_final_train[col] = df_white_noise_train[col] + df_seasonal_train[col] + df_trended_train[col]
        else:
            df_final_train[col] = df_white_noise_train[col] * df_seasonal_train[col] * df_trended_train[col]
        if decomp_type_dict_test[col] == "additive":
            df_final_test[col] = df_white_noise_test[col] + df_seasonal_test[col] + df_trended_test[col]
        else:
            df_final_test[col] = df_white_noise_test[col] * df_seasonal_test[col] * df_trended_test[col]

    ### USE KALMAN FILTER TO SMOOTH ALL DATA (ONLY VISUALIZATION PURPOSE) ###

    smoother_train = KalmanSmoother(component='level_longseason', 
                              component_noise={'level':0.1, 'longseason':0.1}, 
                              n_longseasons=365)
    smoother_train.smooth(df_final_train.T)

    smoother_test = KalmanSmoother(component='level_longseason', 
                              component_noise={'level':0.1, 'longseason':0.1}, 
                              n_longseasons=365)
    smoother_test.smooth(df_final_test.T)

    df_filled = pd.DataFrame(data = smoother_train.smooth_data.T, index = df_train.index, columns= df_train.columns)
    df_filled_test = pd.DataFrame(data = smoother_test.smooth_data.T, index = df_test.index, columns= df_test.columns)


    df_filled.to_csv(r'output\/CompleteDengueTrainDecompKalman_'+str(i)+'.csv', index = True)
    df_filled_test.to_csv(r'output\/CompleteDengueTestDecompKalman_'+str(i)+'.csv', index = True)

Multiplicative seasonality is not appropriate for zero and negative values
additive
Multiplicative seasonality is not appropriate for zero and negative values
additive
Multiplicative seasonality is not appropriate for zero and negative values
additive
Multiplicative seasonality is not appropriate for zero and negative values
additive
Multiplicative seasonality is not appropriate for zero and negative values
additive
multiplicative
multiplicative
multiplicative
multiplicative
multiplicative
Multiplicative seasonality is not appropriate for zero and negative values
additive
multiplicative
Multiplicative seasonality is not appropriate for zero and negative values
additive
multiplicative
multiplicative
multiplicative
multiplicative
multiplicative
multiplicative
Multiplicative seasonality is not appropriate for zero and negative values
additive
multiplicative
multiplicative
multiplicative
multiplicative
Multiplicative seasonality is not appropriate for zero and negative values
additive
mult

additive
multiplicative
multiplicative
multiplicative
multiplicative
multiplicative
multiplicative
multiplicative
Multiplicative seasonality is not appropriate for zero and negative values
additive
Multiplicative seasonality is not appropriate for zero and negative values
additive
Multiplicative seasonality is not appropriate for zero and negative values
additive
Multiplicative seasonality is not appropriate for zero and negative values
additive
Multiplicative seasonality is not appropriate for zero and negative values
additive
Multiplicative seasonality is not appropriate for zero and negative values
additive
Multiplicative seasonality is not appropriate for zero and negative values
additive
Multiplicative seasonality is not appropriate for zero and negative values
additive
Multiplicative seasonality is not appropriate for zero and negative values
additive
Multiplicative seasonality is not appropriate for zero and negative values
additive
Multiplicative seasonality is not appropriate 

multiplicative
multiplicative
multiplicative
multiplicative
multiplicative
multiplicative
multiplicative
multiplicative
multiplicative
multiplicative
multiplicative
multiplicative
multiplicative
multiplicative
multiplicative
multiplicative
multiplicative
multiplicative
Multiplicative seasonality is not appropriate for zero and negative values
additive
Multiplicative seasonality is not appropriate for zero and negative values
additive
Multiplicative seasonality is not appropriate for zero and negative values
additive
Multiplicative seasonality is not appropriate for zero and negative values
additive
Multiplicative seasonality is not appropriate for zero and negative values
additive
Multiplicative seasonality is not appropriate for zero and negative values
additive
Multiplicative seasonality is not appropriate for zero and negative values
additive
Multiplicative seasonality is not appropriate for zero and negative values
additive
Multiplicative seasonality is not appropriate for zero and

additive
Multiplicative seasonality is not appropriate for zero and negative values
additive
Multiplicative seasonality is not appropriate for zero and negative values
additive
Multiplicative seasonality is not appropriate for zero and negative values
additive
Multiplicative seasonality is not appropriate for zero and negative values
additive
Multiplicative seasonality is not appropriate for zero and negative values
additive
Multiplicative seasonality is not appropriate for zero and negative values
additive
Multiplicative seasonality is not appropriate for zero and negative values
additive
Multiplicative seasonality is not appropriate for zero and negative values
additive
Multiplicative seasonality is not appropriate for zero and negative values
additive
Multiplicative seasonality is not appropriate for zero and negative values
additive
Multiplicative seasonality is not appropriate for zero and negative values
additive
Multiplicative seasonality is not appropriate for zero and negative

additive
Multiplicative seasonality is not appropriate for zero and negative values
additive
Multiplicative seasonality is not appropriate for zero and negative values
additive
Multiplicative seasonality is not appropriate for zero and negative values
additive
Multiplicative seasonality is not appropriate for zero and negative values
additive
Multiplicative seasonality is not appropriate for zero and negative values
additive
Multiplicative seasonality is not appropriate for zero and negative values
additive
Multiplicative seasonality is not appropriate for zero and negative values
additive
multiplicative
multiplicative
multiplicative
multiplicative
multiplicative
multiplicative
multiplicative
multiplicative
multiplicative
multiplicative
multiplicative
multiplicative
multiplicative
multiplicative
multiplicative
multiplicative
multiplicative
multiplicative
Multiplicative seasonality is not appropriate for zero and negative values
additive
Multiplicative seasonality is not appropriate for

...

Multiplicative seasonality is not appropriate for zero and negative values
additive
Multiplicative seasonality is not appropriate for zero and negative values
additive
multiplicative
multiplicative
multiplicative
multiplicative
multiplicative
multiplicative
multiplicative
multiplicative
multiplicative
multiplicative
multiplicative
multiplicative
multiplicative
multiplicative
multiplicative
multiplicative
multiplicative
multiplicative
Multiplicative seasonality is not appropriate for zero and negative values
additive
Multiplicative seasonality is not appropriate for zero and negative values
additive
Multiplicative seasonality is not appropriate for zero and negative values
additive
Multiplicative seasonality is not appropriate for zero and negative values
additive
Multiplicative seasonality is not appropriate for zero and negative values
additive
Multiplicative seasonality is not appropriate for zero and negative values
additive
Multiplicative seasonality is not appropriate for zero and

Multiplicative seasonality is not appropriate for zero and negative values
additive
Multiplicative seasonality is not appropriate for zero and negative values
additive
multiplicative
multiplicative
multiplicative
multiplicative
multiplicative
multiplicative
multiplicative
multiplicative
multiplicative
multiplicative
multiplicative
multiplicative
multiplicative
multiplicative
multiplicative
multiplicative
multiplicative
multiplicative
Multiplicative seasonality is not appropriate for zero and negative values
additive
Multiplicative seasonality is not appropriate for zero and negative values
additive
Multiplicative seasonality is not appropriate for zero and negative values
additive
Multiplicative seasonality is not appropriate for zero and negative values
additive
Multiplicative seasonality is not appropriate for zero and negative values
additive
Multiplicative seasonality is not appropriate for zero and negative values
additive
Multiplicative seasonality is not appropriate for zero and

Multiplicative seasonality is not appropriate for zero and negative values
additive
Multiplicative seasonality is not appropriate for zero and negative values
additive
multiplicative
multiplicative
multiplicative
multiplicative
multiplicative
multiplicative
multiplicative
multiplicative
multiplicative
multiplicative
multiplicative
multiplicative
multiplicative
multiplicative
multiplicative
multiplicative
multiplicative
multiplicative
Multiplicative seasonality is not appropriate for zero and negative values
additive
Multiplicative seasonality is not appropriate for zero and negative values
additive
Multiplicative seasonality is not appropriate for zero and negative values
additive
Multiplicative seasonality is not appropriate for zero and negative values
additive
Multiplicative seasonality is not appropriate for zero and negative values
additive
Multiplicative seasonality is not appropriate for zero and negative values
additive
Multiplicative seasonality is not appropriate for zero and

Multiplicative seasonality is not appropriate for zero and negative values
additive
Multiplicative seasonality is not appropriate for zero and negative values
additive
multiplicative
multiplicative
multiplicative
multiplicative
multiplicative
multiplicative
multiplicative
multiplicative
multiplicative
multiplicative
multiplicative
multiplicative
multiplicative
multiplicative
multiplicative
multiplicative
multiplicative
multiplicative
Multiplicative seasonality is not appropriate for zero and negative values
additive
Multiplicative seasonality is not appropriate for zero and negative values
additive
Multiplicative seasonality is not appropriate for zero and negative values
additive
Multiplicative seasonality is not appropriate for zero and negative values
additive
Multiplicative seasonality is not appropriate for zero and negative values
additive
Multiplicative seasonality is not appropriate for zero and negative values
additive
Multiplicative seasonality is not appropriate for zero and

Multiplicative seasonality is not appropriate for zero and negative values
additive
Multiplicative seasonality is not appropriate for zero and negative values
additive
multiplicative
multiplicative
multiplicative
multiplicative
multiplicative
multiplicative
multiplicative
multiplicative
multiplicative
multiplicative
multiplicative
multiplicative
multiplicative
multiplicative
multiplicative
multiplicative
multiplicative
multiplicative
Multiplicative seasonality is not appropriate for zero and negative values
additive
Multiplicative seasonality is not appropriate for zero and negative values
additive
Multiplicative seasonality is not appropriate for zero and negative values
additive
Multiplicative seasonality is not appropriate for zero and negative values
additive
Multiplicative seasonality is not appropriate for zero and negative values
additive
Multiplicative seasonality is not appropriate for zero and negative values
additive
Multiplicative seasonality is not appropriate for zero and